In [1]:
import pandas as pd
import os
# from sentence_transformers import SentenceTransformer
# from sklearn.cluster import AgglomerativeClustering
from tqdm import tqdm

In [2]:
def assign(q,orig:bool=False, cp="delete"):
    global df
    if orig==True:
        df.loc[df.Q==q,"CPCleaned"]=cp
    else:
        df.loc[df.QCleaned==q,"CPCleaned"]=cp
        
def getOriginalQ(cleanedQ):
    global df
    return df[df.QCleaned==cleanedQ].Q.value_counts()

def undo(combinedQ:str):
    global df
    df.loc[df.QCleaned==combinedQ,"QCleaned"]=df.loc[df.QCleaned==combinedQ,"Q"]

def groupSent(sentences, threshold=0.75):
    # Load pre-trained Sentence-BERT model
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(sentences,show_progress_bar=False)
    clustering = AgglomerativeClustering(n_clusters=None, affinity='cosine', linkage='average', distance_threshold=1-threshold)
    clustering.fit(embeddings)
    clusters = {}
    for sentence_id, cluster_id in enumerate(clustering.labels_):
        if cluster_id not in clusters:
            clusters[cluster_id] = []
        clusters[cluster_id].append(sentences[sentence_id])
    return [group for group in clusters.values()]

genSubRule=lambda lol: {x: sorted(t, key=lambda x: len(x))[0] for t in lol for x in t}

def getQNum(qstr):
    global df
    return len(df[df.QCleaned==qstr])

def tabCountry(q):
    global df
    return df[df.QCleaned==q].Country.value_counts()

# with open("grpdQ.txt","r") as txt:
#     grpdQ=eval(txt.read())

def replace(qset, brief=None):
    '''list of questions, replace all with shortest/given'''
    qset=[x.strip() for x in qset]
    global df
    if brief==None:
        brief=sorted(qset,key=lambda x : len(x))[0]
    rplcidx=list(df[df["QCleaned"].isin(qset)].index)
    df.loc[rplcidx,"QCleaned"]=brief
    # df.loc[rplcidx,"cleaned"]=1

def getCountry(q):
    global df
    return df[df.QCleaned==q].Country.unique()

def CPVC(n):
    print(CPCleanedUnique[n])
    return df[df.CPCleaned==CPCleanedUnique[n]].QCleaned.value_counts()

def discard(q):
    global df
    df.drop(df[df.QCleaned==q].index, inplace=True)

def getYear(q):
    return df[df.QCleaned==q].AssesmentDate.str[-4:].unique()

def collapse(qcleaned, country):
    global df
    return df[(df.QCleaned==qcleaned) & (df.Country==country)].Q.value_counts()

def tab(cpnum=None,cp=None):
    assert cpnum!=None or cp!=None, "pass in at least 1 param."
    if cpnum!=None:
        cp=CPCleanedUnique[cpnum]
    print(cp)
    global df
    filtered_df=df[df['CPCleaned']==cp]
    return pd.crosstab(filtered_df['Country'], filtered_df['QCleaned'],margins=True).transpose()

def discardYear(qcleaned, country, year:int=2021):
    global df
    df.loc[(df.QCleaned==qcleaned) & (df.Country==country) & (df.AssesmentDate.str[-4:]==str(year)),"CPCleaned"]="delete"

----

In [4]:
df=pd.read_csv("D://OneDrive - Cornell University//JselZJOB//24SU//w3//r2.csv")

---

In [5]:
tab(cp='wage information, use and deduction')

wage information, use and deduction


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
does the employer keep only one accurate payroll record/ book?,0,0,0,851,0,0,0,851
does the employer keep only one accurate payroll record?,584,2503,161,0,474,122,1802,5646
has the employer made any unauthorized deductions from wages?,584,2503,161,833,474,119,1802,6476
inform workers about wage pay and deduction,584,2503,161,0,474,122,1802,5646
All,1752,7509,483,1684,1422,363,5406,18619


In [6]:
replace(["does the employer keep only one accurate payroll record/ book?","does the employer keep only one accurate payroll record?"])

In [7]:
tab(cp="documentation and protection of young workers")

documentation and protection of young workers


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
does the employer comply with documentation requirements for workers under age of 18?,0,0,0,833,474,0,0,1307
does the employer comply with requirements regarding the certificate of fitness for workers under age 18?,584,0,0,0,0,0,0,584
does the employer cross-check worker's age and documents as stipulated by national law?,0,0,0,0,0,122,0,122
does the employer have a verification system in place to ensure they do not hire child labour in accordance with the national law?,0,0,0,851,0,0,0,851
"does the employer have permission from the parents or legal guardians of workers between the ages of 14 and 16, and approval from the ministry of labor?",0,0,0,0,0,119,0,119
does the employer keep a record of workers under 18 years of age?,584,2237,161,0,0,0,1802,4784
does the employer reliably verify the age of workers prior to hiring?,584,2503,161,0,0,0,1802,5050
does the employer remediate child labour as required under the cba?,0,0,0,0,99,0,0,99
"does the employer require workers under age 18 to provide a medical certificate, and an employment certificate or permit delivered by the director of labor?",0,0,161,0,0,0,0,161


---

In [8]:
discard("does the employer remediate child labour as required under the cba?")
#99 deleted; one unique item asked in some years in Jordan

In [9]:
replace(["does the employer have a verification system in place to ensure they do not hire child labour in accordance with the national law?",
        "does the employer reliably verify the age of workers prior to hiring?",
         "does the employer cross-check worker's age and documents as stipulated by national law?"],"Verify the age of workers before hiring")

In [10]:
replace(["does the employer keep a record of workers under 18 years of age?",
         "does the employer comply with documentation requirements for workers under age of 18?",
         "does the employer have permission from the parents or legal guardians of workers between the ages of 14 and 16, and approval from the ministry of labor?"],
         "Keep a record of workers under age of 18")

In [11]:
discard("does the employer comply with requirements regarding the certificate of fitness for workers under age 18?")
#1 unique item in Bangladesh
discard("does the employer require workers under age 18 to provide a medical certificate, and an employment certificate or permit delivered by the director of labor?")
# 1 unique item in Haiti

In [12]:
tab(cp="overtime")

overtime


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
does the employer comply with daily limits on overtime hours worked?,0,0,0,0,0,0,1802,1802
does the employer comply with limits on overtime hours worked?,584,0,101,0,0,0,0,685
does the employer comply with monthly limits on overtime hours worked?,0,0,0,0,0,0,1555,1555
does the employer comply with the monthly limits on overtime hours worked?,0,0,0,0,0,0,247,247
does the employer comply with yearly limits on overtime hours worked?,0,0,0,0,0,0,1802,1802
does the employer get permission from the labor inspector before suspending weekly rest days?,0,2276,0,0,0,0,0,2276
does the employer get permission from the labour inspector before workers work overtime?,0,2259,80,0,0,122,0,2461
does the employer get written consent from women to work at night?,584,0,0,0,0,0,0,584
does the employer obtain authorization from the department of labour before working on sundays?,0,0,80,0,0,0,0,80


In [13]:
replace(["is overtime work limited to two hours per day?",
         "is overtime limited to three hours per day and nine hours per week?",
         "is overtime limited to 3 hours per day, 14 hours per week?",
         "is overtime in line with legal limits?",
         "does the employer comply with daily limits on overtime hours worked?	",
         "does the employer comply with limits on overtime hours worked?",
         "is overtime worked only for reasons allowed by law?"],"Comply with legal limits on overtime hours")

In [14]:
replace(["does the employer get permission from the labour inspector before workers work overtime?",
         "does the employer get written consent from women to work at night?","Get permission/consent for overtime"])

In [15]:
replace(["does the employer comply with yearly limits on overtime hours worked?","Get permission/consent for overtime"])
# combine one unique item in Vietnam with dissimilar others....

In [16]:
discard("does the employer provide compensatory days off when workers work on weekly rest days?") ##1 unique item in Bangladesh
discard("does the employer schedule overtime on public holidays only under conditions permitted by law?") # 2 strange entries in Nicaragua
discard("does the employer obtain authorization from the department of labour before working on sundays?") #only asked in 2015-2018 in Haiti
discard("does the employer get permission from the labor inspector before suspending weekly rest days?") # 1 item only asked in Camboia

In [17]:
df.loc[(df.QCleaned=="does the employer prepare written instructions on overtime?") & (df.Country=="Indonesia"),"QCleaned"]="Get permission/consent for overtime"
# move 1 unique item in Indonesia to others


In [18]:
for i in ["does the employer comply with monthly limits on overtime hours worked?",
           "does the employer comply with the monthly limits on overtime hours worked?"]:
    discard(i)
#one unique question in Vietnam

In [19]:
discard("does the employer prepare written instructions on overtime?") 

In [20]:
tab(cp="welfare facilities")

welfare facilities


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
adequate washing facilities,584,2503,161,851,474,122,0,4695
does employer provide cups or other sanitary means\nfor drinking water?,0,2368,0,0,0,0,0,2368
does the employer comply with requirements regarding canteen?,0,0,0,0,0,0,1802,1802
does the employer keep food samples for 24 hours?,0,0,0,0,0,0,721,721
does the workplace have a functioning and accessible nursing room?,0,2383,0,0,0,0,0,2383
does the workplace have adequate accessible toilets?,584,2279,161,833,474,122,1802,6255
does the workplace have adequate changing rooms/lockers to meet the needs of workers?,0,0,0,0,474,0,0,474
does the workplace have adequate day care facilities?,584,1230,0,0,474,0,0,2288
does the workplace have adequate hand washing taps?,0,0,0,0,0,0,1802,1802


In [21]:
replace(["does the workplace have adequate hand washing taps?","adequate washing facilities	"])

In [22]:
replace(["does the workplace provide all workers a place to store their clothes?",
         "does the workplace have adequate changing rooms/lockers to meet the needs of workers?",
         "does the workplace have adequate shelters or rest rooms?",
         "does the workplace have all required facilities?"],"Adequate changing rooms/locker")

In [23]:
replace(["does the workplace have an adequate lunch room, and/or canteen?",
         "does the workplace have an adequate eating area?",
         "does the employer comply with requirements regarding canteen?",
         "does employer provide cups or other sanitary means\nfor drinking water?"],
         "Adequate eating area")

In [24]:
replace(["does the workplace have a functioning and accessible nursing room?","Adequate changing rooms/locker"])
#combine 1 unique item in Cambodia with others

In [25]:
discard("does the employer keep food samples for 24 hours?")

In [26]:
df.loc[(df.QCleaned=="is the workplace clean and tidy?"),"CPCleaned"]="working environment"

In [27]:
discard("when provided as in-kind payment, does the employer give workers enough food of decent quality?") # 1 unique in Joradn
discard("does the workplace have adequate day care facilities?") #only asked in Bangladesh, Cambodia, and Jordan


In [28]:
tab(cp="health services and first aid")

health services and first aid


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
do workers have a medical check within the first three months of hiring ?,0,0,161,0,0,0,0,161
do workers have annual medical checks?,584,0,156,834,474,120,0,2168
do workers receive free periodic health checks as required by law?,0,0,161,0,0,0,1802,1963
do workers undergo a medical examination before being employed?,0,2386,0,0,0,2,0,2388
does the employer address safety and health risks to pregnant or nursing workers?,584,2236,161,833,0,122,1802,5738
does the employer comply with requirements on hiv/aids?,584,0,0,833,0,119,1802,3338
does the employer provide adequate first-aid facilities?,0,0,0,851,0,0,0,851
does the employer provide required medical services?,584,0,161,833,474,0,1802,3854
does the workplace have adequate trained first aid officers?,0,0,0,833,0,0,0,833


In [29]:
replace(["has the employer trained workers on first aid and formed a first-aid team?",
         "has the employer provided rescue and first-aid training for workers?",
         "has the employer provided first-aid training for workers?"],
         "Provide first aid training for workers")

In [30]:
replace(["do workers receive free periodic health checks as required by law?",
         "do workers have annual medical checks?",
         "do workers undergo a medical examination before being employed?"],
         "Workers have medical checks as required by law")

In [31]:
df.loc[df.Q.str.contains("annual medical checks"), "QCleaned"]="Workers have annual medical checks Haiti"

In [32]:
df.loc[df.Q=="does the workplace have a functioning hiv/aids committee?", "QCleaned"]="does the employer comply with requirements on hiv/aids?"

In [33]:
getOriginalQ("does the employer comply with requirements on hiv/aids?")

does the workplace have a functioning hiv/aids committee?                    2503
does the employer comply with the law on hiv/aids prevention and control?    1802
does the employer comply with requirements on hiv/aids?                       851
does the employer comply with national policy on hiv/aids?                    566
does the employer comply with requirements on hiv/aids ?                      119
Name: Q, dtype: int64

In [34]:
replace(["does the workplace have an adequately equipped and staffed infirmary?"],
        "does the employer provide required medical services?")
##move 1 in Cambodia to others

In [35]:
for i in ["does the workplace have adequate trained first aid officers?",
"does the employer provide adequate first-aid facilities?",
"do workers have a medical check within the first three months of hiring ?",
"Workers have annual medical checks Haiti"]:
    assign(i)

#unique items only in Haiti or Indonesia

In [36]:
tab(cp="union operations")

union operations


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
employer deducts union dues as requested,584,2240,161,833,474,119,1802,6213
employer provides facility for union,584,2503,0,833,474,0,1802,6196
union has access to workers,584,2503,161,851,474,122,1802,6497
All,1752,7246,322,2517,1422,241,5406,18906


In [37]:
tab(cp="regular hours")

regular hours


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
"do regular daily and weekly working hours exceed legal limits (40 hours per week; 7 hours a day, 6 days a week or 8 hours per day, 5 days a week)?",0,0,0,851,0,0,0,851
"do regular daily or weekly working hours exceed the legal limit (8 hours per day, 48 hours per week)?",584,0,0,0,0,0,0,584
do regular daily or weekly working hours exceed the legal limit?,0,2503,156,0,0,120,0,2779
do regular daily working hours exceed 10 hours?,0,0,0,0,0,0,247,247
do regular daily working hours exceed legal limits?,0,0,5,0,0,0,0,5
do regular work shifts exceed the daily legal limit?,0,0,0,0,0,4,0,4
"do regular working hours exceed 10 hours per day, or 48 hours per week?",0,0,0,0,0,0,1555,1555
do regular working hours exceed 11 hours per day or 48 hours per week?,0,0,0,0,474,0,0,474
do the attendance records reflect the hours actually worked?,0,0,0,835,0,0,0,835


In [38]:
replace(["do regular daily and weekly working hours exceed legal limits (40 hours per week; 7 hours a day, 6 days a week or 8 hours per day, 5 days a week)?",
         "do regular daily or weekly working hours exceed the legal limit (8 hours per day, 48 hours per week)?",
         "do regular daily or weekly working hours exceed the legal limit?",
         "do regular daily working hours exceed legal limits?",
         "do regular working hours exceed 10 hours per day, or 48 hours per week?",
         "do regular daily working hours exceed 10 hours?",
         "do regular working hours exceed 11 hours per day or 48 hours per week?"],
         "Regular daily work hours exceed legal limit")

In [39]:
replace(["do the attendance records reflect the hours actually worked?",
         "do the working time records reflect the hours actually worked?"],
         "Attendance records reflect hours actuallyed worked")

In [40]:
replace(["does the employer comply with daily break periods?",
         "does the employer provide required weekly rest periods?"],
         "Comply with rest periods")

In [41]:
for i in ["does the employer provide required breastfeeding breaks?",
          "does the employer post the factory's working hours (for all shifts, and including break times)?",
          "does the employer comply with shift work requirements?",
          "do workers have at least one day off per week?"]:
    assign(i)

In [42]:
discard("do regular work shifts exceed the daily legal limit?")

In [43]:
tab(cp="contracting procedures")

contracting procedures


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
"do all workers (other than apprentices, badli or casual workers) have a service book?",584,0,0,0,0,0,0,584
do the migrant workers have valid work permits and residence ids?,0,0,0,0,474,0,0,474
do workers have to pay anyone to get a job?,0,2236,0,0,0,0,0,2236
do workers pay any recruitment fees?,0,0,0,833,474,0,0,1307
does the employer comply with legal requirements relating to outsourced workers?,0,0,0,480,0,0,0,480
does the employer comply with legal requirements when subcontracting part of its work to another enterprise?,0,0,0,833,0,0,0,833
does the employer comply with limits on the trial period for apprentices?,584,2241,161,0,0,0,1802,4788
does the employer comply with national law concerning dispatched workers at the workplace?,0,0,0,0,0,0,1802,1802
does the employer comply with requirements concerning home workers?,0,0,0,353,0,0,0,353


In [44]:
replace(["do workers have to pay anyone to get a job?","do workers pay any recruitment fees?"])

In [45]:
replace(["does the employer comply with limits on the trial period for apprentices?",
         "is the probationary period applied only to permanent workers (pkwtt) and for no longer than 3 months?	",
         "is the probationary period limited to 3 months?",
         "Comply with limits on probationary period"])

In [46]:
replace(["does the employer comply with requirements concerning sub-contracted workers at the workplace?",
         "have you found non-compliance with legal requirements for compensation, contracts, osh and/or working time pertaining to non-production workers and/or sub-contracted workers?",
         "does the employer comply with legal requirements relating to outsourced workers?",
         "does the employer comply with national law concerning dispatched workers at the workplace?"],
         "Comply with requirements on subcontracted/outsourced workers")

In [47]:
replace(["limits on the use of fixed term contracts",
         "does the employer consider workers on limited duration contracts who work beyond their term as unlimited duration contract workers?",
         "do all workers (other than apprentices, badli or casual workers) have a service book?	"],
         "Limits on the use of fixed contracts or have contracts")

In [48]:
replace(["do the migrant workers have valid work permits and residence ids?"],
        "Comply with requirements on subcontracted/outsourced workers")
#move 1 in Jordan to others: on requirement for special type of workers

In [49]:
for i in ["has the employer taken sufficient steps to ensure that migrant workers do not pay any unauthorized fees?",
          "does the employer pay for the return trip of migrant workers who have been expelled from the country because of an invalid residence id?",
          "does the employer include the entire period of continuous employment when determining workers' entitlements to maternity leave, attendance bonus, seniority bonus, and/or annual leave?",
          "does the employer comply with the legal requirement relating to compensation for the non-permanent workers?",
          "does the employer comply with requirements for apprentices?",
          "does the employer comply with requirements concerning home workers?",
          "does the employer comply with legal requirements when subcontracting part of its work to another enterprise?"]:
    assign(i)

---
# export:

In [53]:
df.Tags=df.Tags.fillna("")

In [54]:
df.isna().any()

Country              False
FactoryAssessedID    False
AssesmentDate        False
QuestID              False
CP                   False
Q                    False
Finding              False
Tags                 False
disclosed            False
CPCleaned            False
QCleaned             False
id                   False
dtype: bool

In [55]:
df.to_stata("D://r2.dta", version=118)